In [1]:
## Loading Libraries
import pandas as pd
import numpy as np
import sys
import jdatetime as jd
import os
import pickle

# import power_index_calculator as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
## Loading functions
from convert_ar_characters import convert_ar_characters
from nthMax import nth_max
from gini import gini
from find_shapley import find_shapley
from find_banzhaf import find_banzhaf
from gameTheoric_concentration import gameTheoric_concentration
from fill_shapley_banzhaf import fill_shapley_banzhaf

# **Loading Shareholder Data**

In [3]:
# # Yearly
# year = 1397
# file_name_holderdata = "Shareholder"+str(year)+".csv"
# file_name_measures = "Measures"+str(year)+".csv"
# path = r"C:\Users\Mahdi\OneDrive\Master Thesis\Data"

In [16]:
# Monthly
year = 1399
month = 1
file_name_holderdata = "Shareholder"+str(year)+'_'+str(month)+".csv"
file_name_measures = "Measures"+str(year)+'_'+str(month)+".csv"
path = r"C:\Users\Mahdi\OneDrive\Master Thesis\Data\MonthlyShareholder"

In [17]:
## Loading DATA
os.chdir(path)
SDATA = pd.read_csv(file_name_holderdata,index_col=0).drop_duplicates()

In [18]:
# Creating Dataframe for saving concentration mearsurs
CMdf = SDATA.groupby('Symbol',as_index=False).agg({'Id_tse':'first','Industry':'first','percent':'sum',
                                                   'ShareHolder':'count','MarketCap':'first'}).rename(columns=
                                                            {'ShareHolder':'Num_holders','percent':'sum_over1'})
CMdf.reset_index(drop=True,inplace=True)
Orginal_Size = len(CMdf)
print('Number of observed firms in year ',str(year),' is : ',Orginal_Size)

Number of observed firms in year  1399  is :  724


# **Concentration Measures**

## 1- Largest Owner

In [19]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':'max'}).rename(columns={'percent':'Largest_Owner'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left')

## 2- First/Second

In [20]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: max(x)/nth_max(x,nth=2,interval=False)}}).rename(columns={'percent':'First_Second'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('First_Second', '<lambda>'):'First_Second'})

## 3- First/Sumtwofour

In [21]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: max(x)/sum(nth_max(x,nth=[2,4],interval=True))}}).rename(
    columns={'percent':'First_Sumtwofour'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('First_Sumtwofour', '<lambda>'):'First_Sumtwofour'})

## 4- Sumfive

In [22]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: sum(nth_max(x,nth=[1,5],interval=True))}}).rename(columns={'percent':'Sumfive'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Sumfive', '<lambda>'):'Sumfive'})

## 5- Gini

We calculate gini coeficient using Deaton 1997:
$$ \gamma = \frac{N+1}{N-1} - \frac{2}{\mu\times N\times(N-1)}\sum_{i=1}^N{\rho_ix_i}$$

In [23]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: gini(list(x))}}).rename(columns={'percent':'Gini'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Gini', '<lambda>'):'Gini'})

## 6- Herfindhal

In [24]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: sum([(t/100)**2 for t in list(x)])}}).rename(columns={'percent':'Herfindhal'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Herfindhal', '<lambda>'):'Herfindhal'})

## 7 & 8-Shapley-Shubik & Banzhaf

In [25]:

# For refilling
try:
    os.chdir(path)
    CMdf_load = pd.read_csv(file_name_measures)
    CMdf = pd.merge(CMdf,CMdf_load[['Symbol','SSCL', 'SSCO', 'SSDL', 'SSDO', 'BZCL', 'BZCO', 'BZDL']],left_on='Symbol',right_on='Symbol',how='left')
    print('RE-FILL!')
except:
    print('NEW!')
    # For the first time
    # Initiating columns
    CMdf['SSCL'] = np.nan
    CMdf['SSCO'] = np.nan
    CMdf['SSDL'] = np.nan
    CMdf['SSDO'] = np.nan
    CMdf['BZCL'] = np.nan
    CMdf['BZCO'] = np.nan
    CMdf['BZDL'] = np.nan

NEW!


In [26]:
%%time
data = fill_shapley_banzhaf(data = CMdf,SDATA=SDATA,fast_mode = True,time_pnt=True,major_thr = 10)
CMdf = data['CMdf']

print('len(Errors): ',len(data['Errors']))
data['Errors']
[x for x in data['Errors'] if x[2]!= 'Error: request error!']

The symbol   اعتضاد غدیر  mode:  SSCL ,  1  from  2896
The symbol   اعتضاد غدیر  mode:  SSDL ,  2  from  2896
The symbol   اعتضاد غدیر  mode:  BZCL ,  3  from  2896
The symbol   اعتضاد غدیر  mode:  BZDL ,  4  from  2896
The symbol   کاغذ مراغه  mode:  SSCL ,  5  from  2896
The symbol   کاغذ مراغه  mode:  SSDL ,  6  from  2896
The symbol   کاغذ مراغه  mode:  BZCL ,  7  from  2896
The symbol   کاغذ مراغه  mode:  BZDL ,  8  from  2896
The symbol  آ س پ  mode:  SSCL ,  9  from  2896
 It took about  0.06 seconds
The symbol  آ س پ  mode:  SSDL ,  10  from  2896
 It took about  0.86 seconds
The symbol  آ س پ  mode:  BZCL ,  11  from  2896
It took about  0.85 seconds
The symbol  آ س پ  mode:  BZDL ,  12  from  2896
The symbol  آتیمس  mode:  SSCL ,  13  from  2896
The symbol  آتیمس  mode:  SSDL ,  14  from  2896
The symbol  آتیمس  mode:  BZCL ,  15  from  2896
The symbol  آتیمس  mode:  BZDL ,  16  from  2896
The symbol  آرمان  mode:  SSCL ,  17  from  2896
 It took about  0.93 seconds
The symbo

[]

In [27]:
Output_Size = len(CMdf)
print('Orginal Size is ',Orginal_Size,' and output size is: ', Output_Size)
os.chdir(path)
CMdf.to_csv(file_name_measures)

Orginal Size is  724  and output size is:  724
